In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [2]:
spark.sparkContext.defaultParallelism

16

In [3]:
# spark.conf.set("spark.sql.adaptive.enabled", "False")
# spark.conf.set("spark.sql.adaptive.coalescePartitions", "False")

In [4]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

# generating some example data with pandas, will convert to spark df below
df1 = pd.DataFrame({'id':range(0,10*10**5)})
df1['update_date'] = '2025-01-01'
df2 = pd.DataFrame({'id':range(0,10*10**5)})
df2['update_date'] = '2025-02-01'
df3 = pd.DataFrame({'id':range(0,10*10**5)})
df3['update_date'] = '2025-03-01'
dfall = pd.concat([df1,df2,df3])
print(dfall)

            id update_date
0            0  2025-01-01
1            1  2025-01-01
2            2  2025-01-01
3            3  2025-01-01
4            4  2025-01-01
...        ...         ...
999995  999995  2025-03-01
999996  999996  2025-03-01
999997  999997  2025-03-01
999998  999998  2025-03-01
999999  999999  2025-03-01

[3000000 rows x 2 columns]


In [5]:
df = spark.createDataFrame(dfall)

# df.cache()

df.rdd.getNumPartitions()

16

In [6]:
df.orderBy("update_date")\
  .dropDuplicates(subset = ["id"])\
  .filter("update_date == '2025-01-01'")\
  .count()
# 962582

962582

In [9]:
df.orderBy("update_date")\
  .coalesce(1)\
  .dropDuplicates(subset = ["id"])\
  .filter("update_date == '2025-01-01'")\
  .count()

1000000

In [14]:
sc = spark._jsc.sc() 

result1 = sc.getExecutorMemoryStatus().keys() # will print all the executors + driver available

result2 = len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(result1, end ='\n')
print(result2)

Set(EZALOR:55859)
0


In [13]:
sc2 = spark._jsc.sc()
number_of_workers = len([executor.host() for executor in sc2.statusTracker().getExecutorInfos()])-1
print(number_of_workers)

0
